In [1]:
import numpy as np
from numpy import genfromtxt
from numpy import linalg as la

from scipy import interpolate as interp
import scipy.ndimage

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider
from mpl_toolkits.axes_grid1 import make_axes_locatable

import mitsuba; mitsuba.set_variant('scalar_rgb')
import mitsuba.core.spline as sp

# from mitsuba.layer.util.plot.mu_slices import *
# from mitsuba.layer.util.plot.phi_slices import *

from scipy import interpolate

%matplotlib inline
%config InlineBackend.figure_format='svg'

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

| measure_name | measurement_id   | transmission?
|------|------|------|
| goldpaper0 | 4 | no |
| layer_020_004 | 22 | no |
| matte-film-01 | 20 | yes |
| blue-cardboard | 28 | no |
| layer_020_028 | 29 | no |

In [2]:
# Choose measurement id
mid = 4
orientation = 'tb' # 'tb' (top->bottom) or 'bt' (bottom->top) for non-homogeneous measurements with transmission

In [3]:
names = {
    4: "goldpaper0",
    20: "matte-film-01",
    22: "layer_020_004",
    28: "blue-cardboard",
    29: "layer_020_028",
}

has_transmission = {
    4: False,
    20: True,
    22: False,
    28: False,
    29: False,
}

In [4]:
measure_name = names[mid]

# Regular spacing in thetas, up to 85deg
in_theta = np.linspace(85, 0, 20)
in_mu = np.cos(np.deg2rad(in_theta))
in_mu = np.insert(in_mu, 0, 0.0)

m_max = len(in_mu)-1

print('in_mu - in_theta')
print('--------------')
for i in range(len(in_mu)):
    print('%.3f   - %5.2f˚' % (in_mu[i], np.rad2deg(np.arccos(in_mu[i]))))

def incident_to_slice(m):
    return max(0, m-1)

in_mu - in_theta
--------------
0.000   - 90.00˚
0.087   - 85.00˚
0.165   - 80.53˚
0.241   - 76.05˚
0.316   - 71.58˚
0.389   - 67.11˚
0.460   - 62.63˚
0.528   - 58.16˚
0.592   - 53.68˚
0.653   - 49.21˚
0.710   - 44.74˚
0.763   - 40.26˚
0.811   - 35.79˚
0.854   - 31.32˚
0.892   - 26.84˚
0.925   - 22.37˚
0.952   - 17.89˚
0.973   - 13.42˚
0.988   -  8.95˚
0.997   -  4.47˚
1.000   -  0.00˚


In [5]:
def sph(theta, phi):
    sinTheta = np.sin(theta); cosTheta = np.cos(theta);
    sinPhi = np.sin(phi); cosPhi = np.cos(phi);
    return np.array([
        sinTheta * cosPhi,
        sinTheta * sinPhi,
        cosTheta
    ])

def sphInv(w):
    theta = np.arccos(w[2])
    phi = np.arctan2(w[1], w[0])
    return theta, phi

def sph_scalar_theta(theta, phi):
    sinTheta = np.sin(theta); cosTheta = np.cos(theta);
    sinPhi = np.sin(phi); cosPhi = np.cos(phi);
    
    cosTheta = np.ones(sinPhi.shape) * cosTheta
    return np.array([
        sinTheta * cosPhi,
        sinTheta * sinPhi,
        cosTheta
    ])

def normalize(v):
    norm = la.norm(v, axis=(0))
    return v / norm

def halfvector_warp(theta_i, phi_i, theta_h, phi_h, transmission):
    f = np.pi / 180.0
    theta_i = theta_i*f; phi_i = phi_i*f
    theta_h = theta_h*f; phi_h = phi_h*f
    wi = np.array([
        np.sin(theta_i) * np.cos(phi_i),
        np.sin(theta_i) * np.sin(phi_i),
        np.cos(theta_i)
    ])[:, np.newaxis, np.newaxis]
    wh = np.array([
        np.sin(theta_h) * np.cos(phi_h),
        np.sin(theta_h) * np.sin(phi_h),
        np.cos(theta_h)
    ])
    dp = np.sum(wi*wh, axis=0)
    if transmission:
        eta = 1.5
        e = (1/eta)
        sqrt = np.sqrt(1 - e**2 *(1 - dp**2))
        sqrt[sqrt < 0] = 0

        wo = -e*(wi - dp*wh) - wh * sqrt
    else:
        dp = np.sum(wi*wh, axis=0)
        wo = 2 * dp * wh - wi
    f = 180.0 / np.pi
    theta_o = np.arccos(wo[2, ...]) * f
    phi_o = np.arctan2(wo[1, ...], wo[0, ...]) * f 
    phi_o = np.where(phi_o < 0, phi_o + 360, phi_o)
    return theta_o, phi_o

def jacobian(theta_i, phi_i, theta_h, phi_h, transmission):
    if transmission:
        return 1
    
    f = np.pi / 180.0
    theta_i = theta_i*f; phi_i = phi_i*f
    theta_h = theta_h*f; phi_h = phi_h*f
    wi = np.array([
        np.sin(theta_i) * np.cos(phi_i),
        np.sin(theta_i) * np.sin(phi_i),
        np.cos(theta_i)
    ])[:, np.newaxis, np.newaxis]
    wh = np.array([
        np.sin(theta_h) * np.cos(phi_h),
        np.sin(theta_h) * np.sin(phi_h),
        np.cos(theta_h)
    ])
    dp = np.sum(wi*wh, axis=0)
    return 4*np.abs(dp)

In [6]:
def plot_helper(data, extent, title, xlabel, ylabel, log=False):
    plt.figure(figsize=(15, 15))
    ax = plt.gca()
    
    norm = None
    if log:
        norm = LogNorm()
    
    im = ax.imshow(data, extent=extent, cmap='jet', norm=norm)
    
    plt.title(title, size=14)
    plt.xlabel(xlabel, size=14, ha='center', va='top')
    l=plt.ylabel(ylabel, size=14, ha='right', va='center')
    l.set_rotation(0)
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2.5%", pad=0.1)

    plt.colorbar(im, cax=cax)
    
    plt.tight_layout()
    plt.show();

In [7]:
def plot_bsdf(m=0, log=False, fill_values=True, transmission=False):
    alt_set = ''
    ori = ('_%s%s' % (orientation, alt_set)) if has_transmission[mid] else ('%s' % alt_set)
    s = incident_to_slice(m)
    name = "measurements/%s/%d%s_%02d.npz" % (measure_name, mid, ori, s)
    print(name)
    data = np.load(name)["t" if transmission else "r"]
    interpolant = interp.LinearNDInterpolator(data[:, 0:2], data[:, 2])
    if fill_values:
        interpolant_fallback = interp.NearestNDInterpolator(data[:, 0:2], data[:, 2])
        
    phi_o_ = np.linspace(0, 360, 2000)
    if transmission:
        theta_o_ = np.linspace(90, 180, 500)
    else:
        theta_o_ = np.linspace(0, 90, 500)
    phi_o, theta_o = np.meshgrid(phi_o_, theta_o_)
    
    res = interpolant(theta_o, phi_o)
    if fill_values:
        res_fallback = interpolant_fallback(theta_o, phi_o)
        mask = np.isnan(res)
        res[mask] = res_fallback[mask]
        
#     res *= np.cos(np.radians(theta_o))
        
    if transmission:
        extent = [0, 360, 180, 90]
    else:
        extent = [0, 360, 90, 0]
    
    plot_helper(res,
                extent,
                r"$\mu_i = %.2f$,     $\phi_i = 0˚$,     mid = %d" % (in_mu[m], mid),
                r'$\phi_o$',
                r'$\theta_o$',
                log)

interact(plot_bsdf,
         m=IntSlider(min=0, max=m_max, value=0, step=1, continuous_update=False),
         log_scale=False,
         fill_values=True);

interactive(children=(IntSlider(value=0, continuous_update=False, description='m', max=20), Checkbox(value=Fal…

In [8]:
def plot_data_halfvector(m=10, transmission=False, fill_empty_values=True, fill_normal_peak=7):
    ori = ('_%s' % orientation) if has_transmission[mid] else ''
    s = incident_to_slice(m)
    name = "measurements/%s/%d%s_%02d.npz" % (measure_name, mid, ori, s)
    
    data = np.load(name)["t" if transmission else "r"]
    interpolant = interp.LinearNDInterpolator(data[:, 0:2], data[:, 2])
    if fill_empty_values:
        interpolant_nn = interp.NearestNDInterpolator(data[:, 0:2], data[:, 2])
    
    phi_i = 0.0
    mu_i = in_mu[m]
    theta_i = np.rad2deg(np.arccos(mu_i))
    
    wi = sph(np.deg2rad(theta_i), np.deg2rad(phi_i))

    if transmission:
        phi_h_ = np.linspace(-180, 180, 1000)  # Avoid peak at "edge" of data
        theta_h_ = np.linspace(0, 180, 500)
    else:
        phi_h_ = np.linspace(0, 360, 1000)
        theta_h_ = np.linspace(0, 90, 250)
    phi_h, theta_h = np.meshgrid(phi_h_, theta_h_)
    
    theta_o, phi_o = halfvector_warp(theta_i, phi_i, theta_h, phi_h, transmission)
    res = interpolant(theta_o, phi_o)
    
    jac = jacobian(theta_i, phi_i, theta_h, phi_h, transmission)
    
    if fill_empty_values:
        res_nn = interpolant_nn(theta_o, phi_o)
        mask_nn = np.isnan(res)
        res[mask_nn] = res_nn[mask_nn]
    
    # This is an important corner case:
    # For reflection, there is not way to measure the specular peak as it lies direclty on the
    # retroreflection. Thus, we take a slice of an incident mu direction and map it through the
    # halfvector-parameterization to "fill that hole"
    if fill_normal_peak > 0 and not transmission and in_mu[m] == 1.0:
        m_off = m-1
        s_peak = incident_to_slice(m_off)
        
        name = "measurements/%s/%d%s_%02d.npz" % (measure_name, mid, ori, s_peak)
        data_peak = np.load(name)["r"]
        interpolant_peak = interp.LinearNDInterpolator(data_peak[:, 0:2], data_peak[:, 2])
        if fill_empty_values:
            interpolant_peak_nn = interp.NearestNDInterpolator(data_peak[:, 0:2], data_peak[:, 2])

        mu_i_peak = in_mu[m_off]
        theta_i_peak = np.rad2deg(np.arccos(mu_i_peak))
        
        theta_o_peak, phi_o_peak = halfvector_warp(theta_i_peak, phi_i, theta_h, phi_h, transmission)
        res_peak = interpolant_peak(theta_o_peak, phi_o_peak)
        
        if fill_empty_values:
            res_peak_nn = interpolant_peak_nn(theta_o_peak, phi_o_peak)
            mask_peak_nn = np.isnan(res_peak)
            res_peak[mask_peak_nn] = res_peak_nn[mask_peak_nn]

        mask_peak = (theta_o < fill_normal_peak)
        res[mask_peak] = res_peak[mask_peak]
    

    res *= jac
    
    if transmission:
        extent = [-180, 180, 180, 0]
    else:
        extent = [0, 360, 90, 0]
    
    plot_helper(res,
                extent,
                r"$\phi_i = %i˚$,     $\mu_i = %.2f$" % (np.rad2deg(np.arccos(in_mu[m])), mu_i),
                r'$\phi_h$',
                r'$\theta_h$')
    
interact(plot_data_halfvector,
         m=IntSlider(min=0, max=m_max, value=20, step=1, continuous_update=False),
         transmission=False,
         fill_empty_values=True,
         fill_normal_peak=IntSlider(min=0, max=20, value=8, step=1, continuous_update=False));

interactive(children=(IntSlider(value=20, continuous_update=False, description='m', max=20), Checkbox(value=Fa…

In [9]:
n_theta_h = 500
n_phi_h = 2000

brdf = np.zeros((len(in_mu), n_theta_h, n_phi_h))
btdf = np.zeros((len(in_mu), 2*n_theta_h, n_phi_h))

phi_i = 0.0

if has_transmission[mid]:
    total = 2*len(in_mu)
else:
    total = len(in_mu)
current = 0

normal_incidence_fill_degree = 8
    
for i, mu_i in enumerate(in_mu):
    ori = ('_%s' % orientation) if has_transmission[mid] else ''
    s = incident_to_slice(i)
    name = "measurements/%s/%d%s_%02d.npz" % (measure_name, mid, ori, s)
    
    data = np.load(name)["r"]
    interpolant = interp.LinearNDInterpolator(data[:, 0:2], data[:, 2])
    interpolant_nn = interp.NearestNDInterpolator(data[:, 0:2], data[:, 2])
    
    phi_i = 0.0
    mu_i = in_mu[i]
    theta_i = np.rad2deg(np.arccos(mu_i))
    
    wi = sph(np.deg2rad(theta_i), np.deg2rad(phi_i))

    phi_h_ = np.linspace(0, 360, n_phi_h)
    theta_h_ = np.linspace(0, 90, n_theta_h)
    phi_h, theta_h = np.meshgrid(phi_h_, theta_h_)
    
    theta_o, phi_o = halfvector_warp(theta_i, phi_i, theta_h, phi_h, False)
    res = interpolant(theta_o, phi_o)
    
    jac = jacobian(theta_i, phi_i, theta_h, phi_h, False)

    res_nn = interpolant_nn(theta_o, phi_o)
    mask_nn = np.isnan(res)
    res[mask_nn] = res_nn[mask_nn]
    
    if mu_i == 1.0 and normal_incidence_fill_degree > 0:
        i_off = i-1
        s_peak = incident_to_slice(i_off)

        name_peak = "measurements/%s/%d%s_%02d.npz" % (measure_name, mid, ori, s_peak)
        data_peak = np.load(name_peak)["r"]
        interpolant_peak = interp.LinearNDInterpolator(data_peak[:, 0:2], data_peak[:, 2])
        interpolant_peak_nn = interp.NearestNDInterpolator(data_peak[:, 0:2], data_peak[:, 2])
        
        mu_i_peak = in_mu[i_off]
        theta_i_peak = np.rad2deg(np.arccos(mu_i_peak))
        
        theta_o_peak, phi_o_peak = halfvector_warp(theta_i_peak, phi_i, theta_h, phi_h, False)
        res_peak = interpolant_peak(theta_o_peak, phi_o_peak)
        
        res_peak_nn = interpolant_peak_nn(theta_o_peak, phi_o_peak)
        mask_peak_nn = np.isnan(res_peak)
        res_peak[mask_peak_nn] = res_peak_nn[mask_peak_nn]

        mask_peak = (theta_o < normal_incidence_fill_degree)
        res[mask_peak] = res_peak[mask_peak]
    
    jac = jacobian(theta_i, phi_i, theta_h, phi_h, False)
    res *= jac

    brdf[i, :] = res
    
    current += 1
    
    print ("Progress: %.2f%%" % (100*current/total), end="\r")

if has_transmission[mid]:
    for i, mu_i in enumerate(in_mu):
        ori = ('_%s' % orientation) if has_transmission[mid] else ''
        s = incident_to_slice(i)
        name = "measurements/%s/%d%s_%02d.npz" % (measure_name, mid, ori, s)

        data = np.load(name)["t"]
        interpolant = interp.LinearNDInterpolator(data[:, 0:2], data[:, 2])
        interpolant_nn = interp.NearestNDInterpolator(data[:, 0:2], data[:, 2])

        phi_i = 0.0
        mu_i = in_mu[i]
        theta_i = np.rad2deg(np.arccos(mu_i))
        wi = sph(np.deg2rad(theta_i), np.deg2rad(phi_i))

        phi_h_ = np.linspace(-180, 180, n_phi_h)
        theta_h_ = np.linspace(0, 180, 2*n_theta_h)
        phi_h, theta_h = np.meshgrid(phi_h_, theta_h_)

        theta_o, phi_o = halfvector_warp(theta_i, phi_i, theta_h, phi_h, True)
        mu_o = np.cos(np.deg2rad(theta_o))
        res = interpolant(theta_o, phi_o)

        jac = jacobian(theta_i, phi_i, theta_h, phi_h, True)

        res_nn = interpolant_nn(theta_o, phi_o)
        mask_nn = np.isnan(res)
        res[mask_nn] = res_nn[mask_nn]

        jac = jacobian(theta_i, phi_i, theta_h, phi_h, True)
        res *= jac

        res[mu_o*mu_i >= 0.0] = 0

        btdf[i, :] = res

        current += 1

        print ("Progress: %.2f%%" % (100*current/total), end="\r")

In [10]:
@interact(m=(0, m_max, 1))
def plot_data_precomputed(m=10, transmission=False):
    if transmission:
        res = btdf[m, :]
    else:
        res = brdf[m, :]

    plot_helper(res, [0, 360, 90, 0],
                r"$\mu_i = %.2f$,     $\phi_i = 0˚$" % (in_mu[m]),
                r'$\phi_h$',
                r'$\theta_h$')

interactive(children=(IntSlider(value=10, description='m', max=20), Checkbox(value=False, description='transmi…

In [11]:
def interpolate_table(bsdf, nodes, mu_i):
    y = np.zeros((bsdf.shape[1], bsdf.shape[2]))
    
    if mu_i < in_mu[1]:
        # We are in the special region where we don't even have data, fallback to simpler linear interpolation here!
        alpha = (mu_i - in_mu[1]) / (in_mu[2] - in_mu[1])
        y = (1-alpha) * bsdf[1, :, :] + alpha * bsdf[2, :, :]
    elif mu_i < in_mu[len(in_mu)//2]:
        for i in range(len(in_mu)-1):
            if in_mu[i] < mu_i and in_mu[i+1] > mu_i:
                idx0 = i
                idx1 = i + 1
                break
                
        alpha = (mu_i - in_mu[idx0]) / (in_mu[idx1] - in_mu[idx0])
        y = (1-alpha) * bsdf[idx0, :, :] + alpha * bsdf[idx1, :, :]
    else:
        _, offset, weights = sp.eval_spline_weights(nodes, mu_i)
        for i in range(4):
            w = weights[i]
            if w == 0:
                continue
            y += bsdf[offset + i, :, :] * w

    return y

In [12]:
@interact(mu_i=(0.0, 1.0, 0.0001),
          transmission=False)
def plot_data_interpolated(mu_i=0.5, transmission=False):
    if transmission:
        res = interpolate_table(btdf, in_mu, mu_i)
        extent = [-180, 180, 180, 0]
    else:
        res = interpolate_table(brdf, in_mu, mu_i)
        extent = [0, 360, 90, 0]

    plot_helper(res, extent,
                r"$\mu_i = %.2f$,     $\phi_i = 0˚$" % (mu_i),
                r'$\phi_h$',
                r'$\theta_h$',
                False)

interactive(children=(FloatSlider(value=0.5, description='mu_i', max=1.0, step=0.0001), Checkbox(value=False, …

In [24]:
@interact(mu_i=(0.0, 1.0, 0.0001), transmission=False)
def plot_bsdf_interpolated(mu_i=0.05, transmission=False):
    if transmission:
        halfway_parameterization = interpolate_table(btdf, in_mu, mu_i)
    else:
        halfway_parameterization = interpolate_table(brdf, in_mu, mu_i)
    
    theta_i = np.rad2deg(np.arccos(mu_i))
    wi = sph(np.deg2rad(theta_i), np.deg2rad(phi_i))
    
    phi_o_ = np.linspace(0, 360, 2000)
    if transmission:
        theta_o_ = np.linspace(90, 180, 500)
    else:
        theta_o_ = np.linspace(0, 90, 500)
    theta_o, phi_o = np.meshgrid(theta_o_, phi_o_)
    
    wo = sph(np.deg2rad(theta_o), np.deg2rad(phi_o))
    
    if transmission:
        h = -normalize(wi[..., np.newaxis, np.newaxis] + wo*1.5)
    else:
        h = normalize(wi[..., np.newaxis, np.newaxis] + wo)
    
    phi_h_ = np.linspace(0, 360, n_phi_h)
    theta_h_ = np.linspace(0, 90, n_theta_h)
    phi_h, theta_h = np.meshgrid(phi_h_, theta_h_)
    jac = jacobian(theta_i, phi_i, theta_h, phi_h, transmission)
    halfway_parameterization /= jac
    
    theta_h_rad, phi_h_rad = sphInv(h)
    theta_h = np.rad2deg(theta_h_rad); phi_h = np.rad2deg(phi_h_rad)
    if transmission:
        phi_h = (phi_h + 180) / 360 * n_phi_h
    else:
        phi_h = np.where(phi_h < 0, phi_h + 360, phi_h)
        phi_h = phi_h / 360 * n_phi_h
    theta_h = theta_h / 90 * n_theta_h
        
    phi_h_flat = phi_h.reshape(phi_h.shape[0]*theta_h.shape[1], -1)
    theta_h_flat = theta_h.reshape(theta_h.shape[0]*theta_h.shape[1], -1)

    coords = [theta_h_flat, phi_h_flat]
    res = scipy.ndimage.interpolation.map_coordinates(halfway_parameterization, coords, order=1, mode='nearest')

    res = res.reshape(phi_h.shape[0], phi_h.shape[1]).T
    
    if transmission:
        extent = [-180, 180, 180, 90]
    else:
        extent = [-180, 180, 90, 0]

    plot_helper(res, extent,
                r"$\theta_i = %.2f˚$,     $\mu_i = %.2f$" % (theta_i, mu_i),
                r'$\phi_o$',
                r'$\theta_o$',
                False)

interactive(children=(FloatSlider(value=0.05, description='mu_i', max=1.0, step=0.0001), Checkbox(value=False,…

In [25]:
interact(plot_bsdf,
         m=IntSlider(min=0, max=m_max, value=9, step=1, continuous_update=False),
         transmission=False,
         fill_values=True);

interactive(children=(IntSlider(value=9, continuous_update=False, description='m', max=20), Checkbox(value=Fal…

In [28]:
def interpolate_slice(data, nodes, mu_i):
    y = 0

    if mu_i < in_mu[1]:
        # We are in the special region where we don't even have data, fallback to simpler linear interpolation here!
        alpha = (mu_i - in_mu[1]) / (in_mu[2] - in_mu[1])
        y = (1-alpha) * data[1] + alpha * data[2]
    elif mu_i < in_mu[len(in_mu)//2]:
        for i in range(len(in_mu)-1):
            if in_mu[i] < mu_i and in_mu[i+1] > mu_i:
                idx0 = i
                idx1 = i + 1
                break

        alpha = (mu_i - in_mu[idx0]) / (in_mu[idx1] - in_mu[idx0])
        y = (1-alpha) * data[idx0] + alpha * data[idx1]
    else:
        _, offset, weights = sp.eval_spline_weights(nodes, mu_i)
        for i in range(4):
            w = weights[i]
            if w == 0:
                continue
            y += data[offset + i] * w
    return y

In [29]:
@interact(phi_d=(0, np.pi, 0.001),
          scale=True)
def mu_mu_plot(phi_d=0, scale=True):
    n_res = 100

    if has_transmission[mid]:
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(9, 5))
    else:
        fig, (ax1) = plt.subplots(ncols=1, figsize=(9, 5))
    
    ## REFLECTION
    mu_i_ = np.linspace(0, 1, n_res)
    mu_o_ = np.linspace(0, 1, n_res)    # Note difference in mu direction
    mu_i, mu_o = np.meshgrid(mu_i_, mu_o_)
    
    theta_i = np.arccos(mu_i)
    theta_o = np.arccos(mu_o)
    
    phi_i = 0
    phi_o = phi_i + np.pi + phi_d
    phi_o = np.where(phi_o < 0, phi_o + 2*np.pi, phi_o)
        
    # Find halfway angle to lookup brdf values
    wi = sph(theta_i, phi_i)
    wo = sph(theta_o, phi_o)
    
    h = normalize(wi + wo)
    
    theta_h_rad, phi_h_rad = sphInv(h)
    theta_h = np.rad2deg(theta_h_rad); phi_h = np.rad2deg(phi_h_rad)
    phi_h = np.where(phi_h < 0, phi_h + 360, phi_h)
    
    res = np.zeros((n_res, n_res))
    for j in range(n_res):
        for i in range(n_res):
            phi_h_idx = int((phi_h[i, j]) / 360 * (n_phi_h-1))
            theta_h_idx = int(theta_h[i, j] / 90 * (n_theta_h-1))
            
            brdf_slice = brdf[:, theta_h_idx, phi_h_idx]
            
            fr = interpolate_slice(brdf_slice, in_mu, np.abs(mu_i[i, j]))
        
            # Cancel out jacobian of halfvector transform again
            wi_l = wi[:, i, j]
            wh_l = h[:, i, j]
            jac = 4*np.abs(np.dot(wi_l, wh_l))
            fr /= jac
            
#             with np.errstate(divide='ignore'):
#                 inv_cos = 1.0 / np.abs(mu_o[i,j])
#                 if inv_cos == np.inf:
#                     inv_cos = 0
#                 fr *= inv_cos
        
            res[i, j] = fr
    
    if scale:
        res *= np.abs(mu_i * mu_o)
    
    im1 = ax1.imshow(res,
                     cmap='jet',
                     origin='lower',
                     extent=[0, 1, 0, -1],
                     vmin=0,
                     vmax=np.percentile(res, 99))
    
    ax1.set_title('$\mathbf{R}^t$', size=14)
    ax1.set_xlabel('$\mu_i$', size=14, ha='center', va='top')
    l=ax1.set_ylabel('$\mu_o$', size=14, ha='right', va='center')
    l.set_rotation(0)
    
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="2.5%", pad=0.1)
    plt.colorbar(im1, cax=cax)
    
    if has_transmission[mid]:
        ## REFRACTION
        mu_i_ = np.linspace(0, 1, n_res)
        mu_o_ = np.linspace(-1, 0, n_res) # Note difference in mu_o direction
        mu_i, mu_o = np.meshgrid(mu_i_, mu_o_)

        theta_i = np.arccos(mu_i)
        theta_o = np.arccos(mu_o)

        phi_i = 0
        phi_o = phi_i + np.pi + phi_d
        phi_o = np.where(phi_o < 0, phi_o + 2*np.pi, phi_o)

        # Find halfway angle to lookup brdf values
        wi = sph(theta_i, phi_i)
        wo = sph(theta_o, phi_o)

        h = -normalize(wi + wo*1.5)

        theta_h_rad, phi_h_rad = sphInv(h)
        theta_h = np.rad2deg(theta_h_rad); phi_h = np.rad2deg(phi_h_rad)

        res = np.zeros((n_res, n_res))
        for j in range(n_res):
            for i in range(n_res):
                phi_h_idx = int((phi_h[i, j] + 180) / 360 * (n_phi_h-1))
                theta_h_idx = int(theta_h[i, j] / 180 * (2*n_theta_h-1))

                btdf_slice = btdf[:, theta_h_idx, phi_h_idx]

                fr = interpolate_slice(btdf_slice, in_mu, np.abs(mu_i[i, j]))
                
#                 with np.errstate(divide='ignore'):
#                     inv_cos = 1.0 / np.abs(mu_o[i,j])
#                     if inv_cos == np.inf:
#                         inv_cos = 0
#                     fr *= inv_cos

                res[i, j] = np.clip(fr, 0.0, np.inf)

        if scale:
            res *= np.abs(mu_i * mu_o)

        im2 = ax2.imshow(res,
                         cmap='jet',
                         origin='lower',
                         extent=[0, 1, 1, 0],
                         vmin=0,
                         vmax=np.percentile(res, 99))

        ax2.set_title('$\mathbf{T}^{tb}$', size=14)
        ax2.set_xlabel('$\mu_i$', size=14, ha='center', va='top')
        l=ax2.set_ylabel('$\mu_o$', size=14, ha='right', va='center')
        l.set_rotation(0)

        divider = make_axes_locatable(ax2)
        cax = divider.append_axes("right", size="2.5%", pad=0.1)
        plt.colorbar(im2, cax=cax)
    
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.0, description='phi_d', max=3.141592653589793, step=0.001), Checkbox…

In [30]:
def eval_bsdf(mu_i, mu_o, phi_d):
    transmission = (mu_i*mu_o > 0)
    
    mu_o = -mu_o # Difference in parameterization
    inv_cos = 1.0 / np.abs(mu_o)
    
    theta_i = np.arccos(mu_i)
    theta_o = np.arccos(mu_o)
    
    phi_i = 0
    phi_o = phi_i + np.pi + phi_d
    
    # Find halfway angle to lookup brdf values
    wi = sph_scalar_theta(theta_i, phi_i)
    wo = sph_scalar_theta(theta_o, phi_o)
    
    if transmission:
        h = -normalize(wi[..., np.newaxis] + wo*1.5)
    else:
        h = normalize(wi[..., np.newaxis] + wo)
    
    theta_h_rad, phi_h_rad = sphInv(h)
    theta_h = np.rad2deg(theta_h_rad); phi_h = np.rad2deg(phi_h_rad)
    if not transmission:
        phi_h = np.where(phi_h < 0, phi_h + 360, phi_h)
    
    n = len(phi_d)
    res = np.zeros(n)
    for i in range(n):
        if transmission:
            phi_h_idx = int((phi_h[i] + 180) / 360 * (n_phi_h-1))
            theta_h_idx = int(theta_h[i] / 180 * (2*n_theta_h-1))
            bsdf_slice = btdf[:, theta_h_idx, phi_h_idx]
        else:
            phi_h_idx = int((phi_h[i]) / 360 * (n_phi_h-1))
            theta_h_idx = int(theta_h[i] / 90 * (n_theta_h-1))
            bsdf_slice = brdf[:, theta_h_idx, phi_h_idx]
            
        fr = interpolate_slice(bsdf_slice, in_mu, mu_i)
        
        res[i] = fr
        
    if not transmission: 
        jac = 4*np.abs(np.sum(wi[..., np.newaxis]*h, axis=0))
        res /= jac
        
#     res *= inv_cos
    
    return res

In [31]:
def fourier(x, coeffs):
        x = np.asarray(x)
        indices = np.arange(-(len(coeffs)//2), len(coeffs)//2+1)
        return np.sum(np.exp(1j*x[..., np.newaxis] * indices) * coeffs, axis=-1)

In [32]:
@interact(mu_i=(0, 1, 0.001),
          mu_o=(-1, +1, 0.001),
          md=(1, 1001, 2),
          scale_energy=False)
def phi_phi_plot_data(mu_i=0.5, mu_o=-0.5, md=5,scale_energy=False):
    n_res = 1000
    
    phi_d = np.linspace(-np.pi, np.pi, n_res)
    fr = eval_bsdf(mu_i, mu_o, phi_d)
    
    if scale_energy:
        fr *= np.abs(mu_i * mu_o)
        
    samples = np.linspace(0, 2*np.pi, md, endpoint=False)
    values = eval_bsdf(mu_i, mu_o, samples)
    if scale_energy:
        values *= np.abs(mu_i * mu_o)
    coeffs = np.real(mitsuba.layer.fftw_transform_r2c(values))
    
    fs = fourier(phi_d, coeffs)
    
    plt.figure(figsize=(5, 5))
    ax = plt.gca()
        
    plt.plot(phi_d, fr, label='BSDF')
    plt.plot(phi_d, fs, label='Fourier')

    plt.title('Azimuthal slice', size=14)
    plt.xlabel('$\phi_d$', size=14, ha='center', va='top')
    l=plt.ylabel('$f_r$', size=14, ha='right', va='center')
    l.set_rotation(0)
    
    plt.legend()
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='mu_i', max=1.0, step=0.001), FloatSlider(value=-0.5,…